In [ ]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('fast-reid/')

from demo.predictor import FeatureExtractionDemo
from fastreid.config import get_cfg


def setup_cfg(config_file, weights, device="cuda"):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights
    cfg.MODEL.DEVICE = device if torch.cuda.is_available() else "cpu"
    cfg.freeze()
    return cfg

def extract_features_from_folder(img_folder, predictor, cfg, save_path="reid_features.npz"):
    dataset_embeddings = {}
    for fname in tqdm(os.listdir(img_folder)):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(img_folder, fname)
            img = cv2.imread(img_path)
            feat = predictor.run_on_image(img)

            dataset_embeddings[fname] = feat

    return dataset_embeddings



In [ ]:
# CONFIG_FILE = "fast-reid/configs/Market1501/bagtricks_S50.yml"
# MODEL_WEIGHTS = "market_bot_S50.pth"
CONFIG_FILE = "fast-reid/configs/Market1501/sbs_R101-ibn.yml" 
MODEL_WEIGHTS = "market_sbs_R101-ibn.pth"

IMG_FOLDER = "/home/gfuhr/projects/data/immediate_action/people_v1_cropped"
SAVE_PATH = "reid_features_sbs_r101.npz"
DEVICE = "cuda"
cfg = setup_cfg(CONFIG_FILE, MODEL_WEIGHTS, DEVICE)
predictor = FeatureExtractionDemo(cfg)
dataset_embeddings = extract_features_from_folder(IMG_FOLDER, predictor, cfg, SAVE_PATH)

np.savez(SAVE_PATH, **dataset_embeddings)
